In [1]:
import pandas as pd
import numpy as np 
import yfinance as yf
import matplotlib.pyplot as plt
import pickle 

data = pd.read_pickle("stocksData.pkl")
data ={i[0]:i[1] for i in data if type(data[1]) != int}

In [2]:
tickers = [i for i in data if type(data[i]) != int]
df = pd.DataFrame(index = tickers, columns = ['BreakOuts'])


In [1]:
def break_out(stock_data,high_back = 242 , size_back = 60):
    stock_data.dropna(inplace = True)
    stock_data['max'] = stock_data['Adj Close'].rolling(high_back).max().dropna() #60
    stock_data['breakout'] = np.where(stock_data['Adj Close'] == stock_data['max'],1,0)
    stock_data['size'] = abs((stock_data['Open']- stock_data['Close'])/stock_data['Close'] * 100)
    stock_data.dropna(inplace = True)
    stock_data['Average size'] = stock_data['size'].rolling(size_back).mean() #40
    stock_data['size signal'] = np.where(stock_data['size']>stock_data['Average size'],1,0)
    stock_data['average volume'] = stock_data['Volume'].rolling(size_back).mean()#40
    stock_data['volume signal'] = np.where(stock_data['average volume']< stock_data['Volume'],1,0)
    stock_data['buy signal'] = np.where((stock_data['breakout']==1) & (stock_data['size signal'] ==1) & (stock_data['volume signal'] == 1),1,0)
    df = stock_data[stock_data['buy signal']==1]
    try:
        return df.index[-1]
    except:
        return np.nan

In [4]:
for stock in tickers:
    df.at[stock,'BreakOuts'] = break_out(data[stock])
    

In [5]:
test = df.sort_values( by = ['BreakOuts'], ascending = False)
print(test[:100].to_string())

                 BreakOuts
NABZY  2022-02-14 00:00:00
GFI    2022-02-14 00:00:00
TSEM   2022-02-14 00:00:00
ITOCY  2022-02-14 00:00:00
SHBI   2022-02-14 00:00:00
SEE    2022-02-14 00:00:00
SBLK   2022-02-14 00:00:00
ITCI   2022-02-14 00:00:00
DLTR   2022-02-14 00:00:00
ECPG   2022-02-14 00:00:00
HUN    2022-02-14 00:00:00
APTS   2022-02-14 00:00:00
HSIC   2022-02-14 00:00:00
NXRT   2022-02-14 00:00:00
USAC   2022-02-14 00:00:00
LXU    2022-02-08 00:00:00
CCK    2022-02-07 00:00:00
ARTW   2022-02-07 00:00:00
NMFC   2022-02-07 00:00:00
PAC    2022-02-07 00:00:00
PBR    2022-02-07 00:00:00
RE     2022-02-07 00:00:00
SNEX   2022-02-07 00:00:00
SQM    2022-02-07 00:00:00
NLOK   2022-02-07 00:00:00
NGG    2022-02-07 00:00:00
OMC    2022-02-07 00:00:00
AA     2022-02-07 00:00:00
MOS    2022-02-07 00:00:00
MCRI   2022-02-07 00:00:00
CHT    2022-02-07 00:00:00
CLMT   2022-02-07 00:00:00
COOP   2022-02-07 00:00:00
CPHC   2022-02-07 00:00:00
CPRX   2022-02-07 00:00:00
DE     2022-02-07 00:00:00
E

In [13]:
def pct_ret(start_price,end_price):
    delta = end_price - start_price
    percent = (delta/start_price)*100
    return round(percent,2)

def get_returns(data,start_date):
    delta = np.arange(1,60).tolist()
    rets = data.loc[start_date:].pct_change().cumsum()[:61]
    rets = [v for c,v in rets if c in delta]
    return rets


In [11]:
data['AAPL'].columns+ np.arange(1,60).tolist()

ValueError: operands could not be broadcast together with shapes (15,) (59,) 

In [9]:
data['AAPL'][].apply(lambda x: x.index)

,Open,High,Low,Close,Adj Close,Volume,max,breakout,size,Average size,size signal,average volume change,volume change,volume signal,buy signal
Date,,,,,,,,,,,,,,,
1981-02-09,0.122768,0.122768,0.121652,0.121652,0.095508,16755200,0.126175,0,0.917371,NaN,0,NaN,NaN,0,0
1981-02-10,0.121652,0.122210,0.121652,0.121652,0.095508,18345600,0.126175,0,0.000000,NaN,0,NaN,0.094920,0,0
1981-02-11,0.118304,0.118304,0.117746,0.117746,0.092441,13843200,0.126175,0,0.473898,NaN,0,NaN,-0.245421,0,0
1981-02-12,0.117188,0.117188,0.116629,0.116629,0.091564,14560000,0.126175,0,0.479299,NaN,0,NaN,0.051780,0,0
1981-02-13,0.114955,0.114955,0.113839,0.113839,0.089374,11155200,0.126175,0,0.980332,NaN,0,NaN,-0.233846,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-17,169.929993,173.470001,169.690002,171.139999,171.139999,195432700,179.449997,0,0.707027,2.178495,0,0.063284,0.301273,1,0
2021-12-20,168.279999,170.580002,167.460007,169.750000,169.750000,107499100,179.449997,0,0.865980,1.929030,0,0.053068,-0.449943,0,0
2021-12-21,171.559998,173.199997,169.119995,172.990005,172.990005,91185900,179.449997,0,0.826642,1.556581,0,0.052740,-0.151752,0,0


In [26]:
data['AAPL'][-30:]['Volume'].pct_change().describe()

count    29.000000
mean      0.053561
std       0.297151
min      -0.449943
25%      -0.126377
50%      -0.028292
75%       0.145903
max       0.961145
Name: Volume, dtype: float64